In [4]:
import pandas as pd
import math

# Load data
df = pd.read_csv('/content/simpledata (1).csv')

# Calculate entropy
def entropy(data):
    counts = data['PlayTennis'].value_counts()
    probs = counts / len(data)
    return -sum(probs * probs.apply(lambda p: math.log2(p) if p > 0 else 0))

# Calculate info gain for an attribute
def info_gain(data, attribute):
    total_entropy = entropy(data)
    weighted_entropy = sum((len(data[data[attribute] == value]) / len(data)) * entropy(data[data[attribute] == value]) for value in data[attribute].unique())
    return total_entropy - weighted_entropy

# Build the tree
def build_tree(data):
    if len(data['PlayTennis'].unique()) == 1:
        return data['PlayTennis'].iloc[0]

    best_attr = max(data.columns[:-1], key=lambda attr: info_gain(data, attr))
    tree = {best_attr: {}}

    for value in data[best_attr].unique():
        subset = data[data[best_attr] == value]
        tree[best_attr][value] = build_tree(subset.drop(columns=[best_attr]))

    return tree

# Classify a new sample
def classify(tree, sample):
    if isinstance(tree, dict):
        attribute = list(tree.keys())[0]
        value = sample[attribute]
        return classify(tree[attribute].get(value), sample)
    else:
        return tree

# Build and print tree
tree = build_tree(df)
print(tree)

# Classify a new sample
new_sample = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'Normal', 'Wind': 'Weak'}
print(classify(tree, new_sample))

{'Outlook': {'Sunny': 'No', 'Overcast': 'Yes', 'Rainy': {'Temperature': {'Mild': 'Yes', 'Cool': 'No'}}}}
No
